In [ ]:
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
import importlib
import numpy as np
import polars as pl

from LuStCodeSnippets_py.Astronomy import Rates

importlib.reload(Rates)

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

np.float64(0.7)

In [ ]:
#%%specifications

df_rates = pl.from_dicts([
    dict(setting="Grayling2023 sncc",  nobs=50+69, survey="DES-SN",             rate=7.0e-5   * cosmo.h**3/(u.Mpc**3 * u.yr), z_min=0.045, z_max=0.25, source="Pessi2025, Dahlen2012", note=""),
    dict(setting="Grayling2023 snibc", nobs=50,    survey="DES-SN",             rate=1.266e-5 * cosmo.h**3/(u.Mpc**3 * u.yr), z_min=0.045, z_max=0.25, source="Pessi2025",             note="(Ibc + Ib + Ic + Ic-BL) -> Grayling2023 includes all stripped envelope SNe in SNIbc"),
    dict(setting="Grayling2023 snii",  nobs=69,    survey="DES-SN",             rate=4.3e-5   * cosmo.h**3/(u.Mpc**3 * u.yr), z_min=0.045, z_max=0.25, source="Pessi2025",             note=""),
    dict(setting="Moeller2024 snia",   nobs=2298,  survey="DES-SN",             rate=2.43e-5  * cosmo.h**3/(u.Mpc**3 * u.yr), z_min=0.07,  z_max=1.14, source="Frohmaier2019",         note=""),
    dict(setting="Moeller2024 snia",   nobs=2298,  survey="DES-SN",             rate=(np.array([2.1,3.0,3.5,3.6,4.8,5.5,5.9]) * 1e-5).mean() * cosmo.h**3/(u.Mpc**3 * u.yr), z_min=0.07,  z_max=0.9, source="Perrett2012", note="z_max based on DES detection limit for SNIa"),
    dict(setting="Moeller2024 snia",   nobs=2298,  survey="DES-SN (approx)",    rate=2.43e-5  * cosmo.h**3/(u.Mpc**3 * u.yr), z_min=0.07,  z_max=1.14, source="Frohmaier2019",         note="fov_eff in degrees => approximation used; multiplied by np.sqrt(10) because 10 fields in total"),
    dict(setting="Pessi2025 sncc",      nobs=173,   survey="ASSA-SN",            rate=7.0e-5   * cosmo.h**3/(u.Mpc**3 * u.yr), z_min=0.001, z_max=0.02, source="Pessi2025",             note="used ceil(median redshift) as z_max"),
    dict(setting="Pessi2025 snibc",     nobs=2,     survey="ASSA-SN",            rate=0.033e-5 * cosmo.h**3/(u.Mpc**3 * u.yr), z_min=0.001, z_max=0.02, source="Pessi2025",             note="used ceil(median redshift) as z_max"),
    dict(setting="Pessi2025 snii",      nobs=104,   survey="ASSA-SN",            rate=4.3e-5   * cosmo.h**3/(u.Mpc**3 * u.yr), z_min=0.001, z_max=0.02, source="Pessi2025",             note="used ceil(median redshift) as z_max"),
])
# print(df_rates)

#survey specifications
df_surveys = pl.from_dicts([
    dict(survey="DES-SN",           source="Smith2020", fov_eff=23.0 * u.deg**2,        dt=(5.5/12 * 5*u.yr)),
    dict(survey="DES-SN (approx)",  source="Smith2020", fov_eff=np.sqrt(10)*2.7 * u.deg,  dt=(5.5/12 * 5*u.yr)),
    dict(survey="ASSA-SN",          source="Pessi2025", fov_eff=(4*np.pi * (180/np.pi)**2 * u.deg**2 *1/(1-np.sin(15*np.pi/180))), dt=((2018-2014.5) *u.yr)),
])
# print(df_surveys)

df = df_rates.join(df_surveys, on="survey", how="inner", suffix="_survey")
df

setting,nobs,survey,rate,z_min,z_max,source,note,source_survey,fov_eff,dt
str,i64,str,object,f64,f64,str,str,str,object,object
"""Grayling2023 sncc""",119,"""DES-SN""",2.4009999999999992e-05 1 / (yr Mpc3),0.045,0.25,"""Pessi2025, Dahlen2012""","""""","""Smith2020""",23.0 deg2,2.2916666666666665 yr
"""Grayling2023 snibc""",50,"""DES-SN""",4.34238e-06 1 / (yr Mpc3),0.045,0.25,"""Pessi2025""","""(Ibc + Ib + Ic + Ic-BL) -> Gra…","""Smith2020""",23.0 deg2,2.2916666666666665 yr
"""Grayling2023 snii""",69,"""DES-SN""",1.4748999999999997e-05 1 / (yr Mpc3),0.045,0.25,"""Pessi2025""","""""","""Smith2020""",23.0 deg2,2.2916666666666665 yr
"""Moeller2024 snia""",2298,"""DES-SN""",8.334899999999999e-06 1 / (yr Mpc3),0.07,1.14,"""Frohmaier2019""","""""","""Smith2020""",23.0 deg2,2.2916666666666665 yr
"""Moeller2024 snia""",2298,"""DES-SN""",1.3915999999999997e-05 1 / (yr Mpc3),0.07,0.9,"""Perrett2012""","""z_max based on DES detection l…","""Smith2020""",23.0 deg2,2.2916666666666665 yr
"""Moeller2024 snia""",2298,"""DES-SN (approx)""",8.334899999999999e-06 1 / (yr Mpc3),0.07,1.14,"""Frohmaier2019""","""fov_eff in degrees => approxim…","""Smith2020""",8.538149682454625 deg,2.2916666666666665 yr
"""PESS2025 sncc""",173,"""ASSA-SN""",2.4009999999999992e-05 1 / (yr Mpc3),0.001,0.02,"""Pessi2025""","""used ceil(median redshift) as …","""Pessi2025""",55658.42049344808 deg2,3.5 yr
"""PESS2025 snibc""",2,"""ASSA-SN""",1.1318999999999998e-07 1 / (yr Mpc3),0.001,0.02,"""Pessi2025""","""used ceil(median redshift) as …","""Pessi2025""",55658.42049344808 deg2,3.5 yr
"""PESS2025 snii""",104,"""ASSA-SN""",1.4748999999999997e-05 1 / (yr Mpc3),0.001,0.02,"""Pessi2025""","""used ceil(median redshift) as …","""Pessi2025""",55658.42049344808 deg2,3.5 yr


In [6]:
df_res = []
for row in df.iter_rows(named=True):

    npred = Rates.number_from_rate(rate=row["rate"], z=row["z_max"], dt=row["dt"], fov_eff=row["fov_eff"], z_min=row["z_min"], cosmo=cosmo)
    df_res.append(
        dict(setting=row["setting"], nobs=row["nobs"], npred=npred)
    )

df_res = pl.from_dicts(df_res)
df_res

using approximation because `fov_eff.unit` is `u.deg`. If you want no approximation provide `fov_eff` as an area


setting,nobs,npred
str,i64,object
"""Grayling2023 sncc""",119,130.77228105267966
"""Grayling2023 snibc""",50,23.65110111609893
"""Grayling2023 snii""",69,80.33154407521752
"""Moeller2024 snia""",2298,2134.3169190474623
"""Moeller2024 snia""",2298,2118.1837734605533
"""Moeller2024 snia""",2298,1158.3005345701276
"""PESS2025 sncc""",173,294.39227878187546
"""PESS2025 snibc""",2,1.387849314257413
"""PESS2025 snii""",104,180.84097125172354
